# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')
df.columns = df.columns.str.replace(' ', '_').str.lower()
print (df.shape)
df.head()

(10910, 26)


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [2]:
df1 = df.groupby('response')
df['response'].nunique() == len(df1)

True

In [3]:
df1.size()

response
No     8813
Yes    1466
dtype: int64

In [4]:
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]
filtered_df.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [5]:
df_tca = df[df['response'] == 'Yes']
average_claim_amount = df_tca.groupby(['policy_type', 'gender'])['total_claim_amount'].mean()
average_claim_amount

policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [6]:
df_st = df.groupby('state')['customer'].count()
df_st
df_st1 = df_st[df_st >500]
df_st1

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
df_egma = df.groupby(['education', 'gender'])['customer_lifetime_value'].max().round(1)
df_egmi = df.groupby(['education', 'gender'])['customer_lifetime_value'].min().round(1)
df_egme = df.groupby(['education', 'gender'])['customer_lifetime_value'].mean().round(1)
print (df_egma, df_egmi,df_egme )

education             gender
Bachelor              F         73226.0
                      M         67907.3
College               F         61850.2
                      M         61134.7
Doctor                F         44856.1
                      M         32677.3
High School or Below  F         55277.4
                      M         83325.4
Master                F         51016.1
                      M         50568.3
Name: customer_lifetime_value, dtype: float64 education             gender
Bachelor              F         1904.0
                      M         1898.0
College               F         1898.7
                      M         1918.1
Doctor                F         2395.6
                      M         2267.6
High School or Below  F         2144.9
                      M         1941.0
Master                F         2417.8
                      M         2272.3
Name: customer_lifetime_value, dtype: float64 education             gender
Bachelor              F        

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [8]:
df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')

In [13]:
df_sm = df.groupby(['state', 'months_since_policy_inception'])['number_of_policies'].sum()
df_sm = df.pivot_table(index = 'state', columns = 'months_since_policy_inception', values = ['number_of_policies'], aggfunc='sum')
df_sm

number_of_policies                              \
months_since_policy_inception                 0    1    2    3    4   5   6    
state                                                                          
Arizona                                       53   44   36   67   43  54  70   
California                                   189  107   70  114  143  80  92   
Nevada                                        18   14   24   24   24  14   6   
Oregon                                        54   99  123   91   67  88  73   
Washington                                    25   25   39   29   16  15  11   

                                            ...                               \
months_since_policy_inception  7   8    9   ...  90   91  92  93  94  95  96   
state                                       ...                                
Arizona                        27  61   44  ...  40   55  47  53  92  33  17   
California                     87  93  143  ...  87   50  60  63  70  63  76   
Nevada                         32  51   37  ...  34   27   2  43   9  38  12   
Oregon                         88  83   95  ...  54  119  97  82  54  57  62   
Washington                     17  21    5  ...  55   18  20  26  16  23  19   

                                            
months_since_policy_inception  97  98   99  
state                                       
Arizona                        18  34   70  
California                     50  58  117  
Nevada                         21  20   15  
Oregon                         67  50   59  
Washington                      5  28   19  

[5 rows x 100 columns]

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here